# Data Cleaning
封装了一个函数，专门初始化数据，返回一个DataFrame和dict，具体内容见下：

In [1]:
# Data Cleaning Implementation
import pandas as pd


def data_clean():
    jet = pd.read_table('train_data.txt', sep=' ', header=None)
    jet.drop([26, 27], axis=1, inplace=True)
    jet_index_1 = jet.iloc[:, 0].tolist()
    jet_index_2 = jet.iloc[:, 1].tolist()
    jet_index = pd.MultiIndex.from_arrays(
        [jet_index_1, jet_index_2], names=['id', 'flight'])
    jet.set_index(jet_index, inplace=True)
    jet.drop([0, 1], axis=1, inplace=True)
    sensors = '''Altitude
    Mach #
    Power Setting (TRA)
    12 Total temperature at fan inlet°R
    T24 Total temperature at LPC outlet °R
    T30 Total temperature at HPC outlet °R
    T50 Total temperature at LPT outlet °R
    P2 Pressure at fan inlet psia
    P15 Total pressure in bypass-duct psia
    P30 Total pressure at HPC outlet psia
    Nf Physical fan speed rpm
    Nc Physical core speed rpm
    epr Engine pressure ratio (P50/P2) -
    Ps30 Static pressure at HPC outlet psia
    phi Ratio of fuel flow to Ps30 pps/psi
    NRf Corrected fan speed rpm
    NRc Corected core speed rpm
    BPR Bypass Ratio-
    farB Bumer fuel-air ratio
    htBleed (Bleed Enthalpy)
    Nf_dmd Demanded fan speed rpm
    PCNFR_dmd Demanded corre dted fan speed rpm
    w31 HPT cool ant bleed lbm/s
    w32 LPT coolant bleed lbm/s'''
    sensors = sensors.split(sep='\n')
    num_col = list(range(1, 25))
    jet.columns = num_col
    sensors_dict = dict(zip(num_col, sensors))
    return jet, sensors_dict

In [2]:
jet, sensor_dict = data_clean()

In [3]:
jet.head()

1       2      3       4       5        6        7     8   \
id flight                                                                   
1  1       34.9983  0.8400  100.0  449.44  555.32  1358.61  1137.23  5.48   
   2       41.9982  0.8408  100.0  445.00  549.90  1353.22  1125.78  3.91   
   3       24.9988  0.6218   60.0  462.54  537.31  1256.76  1047.45  7.05   
   4       42.0077  0.8416  100.0  445.00  549.51  1354.03  1126.38  3.91   
   5       25.0005  0.6203   60.0  462.54  537.07  1257.71  1047.93  7.05   

             9       10   ...        15       16       17       18    19   20  \
id flight                 ...                                                   
1  1       8.00  194.64   ...    183.06  2387.72  8048.56   9.3461  0.02  334   
   2       5.71  138.51   ...    130.42  2387.66  8072.30   9.3774  0.02  330   
   3       9.02  175.71   ...    164.22  2028.03  7864.87  10.8941  0.02  309   
   4       5.71  138.46   ...    130.72  2387.61  8068.66   9.3528  0.02  329   
   5       9.03  175.05   ...    164.31  2028.00  7861.23  10.8963  0.02  309   

             21      22     23      24  
id flight                               
1  1       2223  100.00  14.73  8.8071  
   2       2212  100.00  10.41  6.2665  
   3       1915   84.93  14.08  8.6723  
   4       2212  100.00  10.59  6.4701  
   5       1915   84.93  14.13  8.5286  

[5 rows x 24 columns]

In [18]:
sensor_dict

{1: 'Altitude',
 2: '    Mach #',
 3: '    Power Setting (TRA)',
 4: '    12 Total temperature at fan inlet°R',
 5: '    T24 Total temperature at LPC outlet °R',
 6: '    T30 Total temperature at HPC outlet °R',
 7: '    T50 Total temperature at LPT outlet °R',
 8: '    P2 Pressure at fan inlet psia',
 9: '    P15 Total pressure in bypass-duct psia',
 10: '    P30 Total pressure at HPC outlet psia',
 11: '    Nf Physical fan speed rpm',
 12: '    Nc Physical core speed rpm',
 13: '    epr Engine pressure ratio (P50/P2) -',
 14: '    Ps30 Static pressure at HPC outlet psia',
 15: '    phi Ratio of fuel flow to Ps30 pps/psi',
 16: '    NRf Corrected fan speed rpm',
 17: '    NRc Corected core speed rpm',
 18: '    BPR Bypass Ratio-',
 19: '    farB Bumer fuel-air ratio',
 20: '    htBleed (Bleed Enthalpy)',
 21: '    Nf_dmd Demanded fan speed rpm',
 22: '    PCNFR_dmd Demanded corre dted fan speed rpm',
 23: '    w31 HPT cool ant bleed lbm/s',
 24: '    w32 LPT coolant bleed lbm/s'}

# Data Visualization
可见少部分传感器是二元的，大部分是连续分布的，传感器的参数也反映了不同的飞行模式。

In [5]:
jet.hist(bins=20,figsize=(18,16));

3,16,19,22传感器都只返回两个值

# Clustering
## 1. 对全集聚类
使用k-means进行一个简单的聚类（**kmeans很可能不够合理**），分开六种飞行模式，方法是直接对训练数据总集（对总集聚类也可能不够合理）进行聚类。

In [6]:
from sklearn import cluster

In [7]:
k_cluster = cluster.KMeans(n_clusters=6) 

In [8]:
import numpy as np

In [9]:
def max_min_normalize(df):  # 这里用的极值标准化 不知道用正态标准化会怎么样
    return (df-df.min())/(df.max()-df.min())

In [10]:
train_data = np.array(max_min_normalize(jet))

In [11]:
k_cluster.fit(train_data)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=6, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [12]:
set(k_cluster.labels_) # 验证模式有几个

{0, 1, 2, 3, 4, 5}

In [13]:
pd.DataFrame(k_cluster.cluster_centers_)  # 聚类中心，没什么用

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.000036,0.000587,1.000000e+00,1.000000,0.983106,0.939161,0.926903,1.000000,0.999989,0.994131,...,0.995015,0.993440,0.698107,0.039011,1.000000e+00,0.939568,1.000000e+00,1.000000e+00,0.982081,0.982932
1,0.937576,0.998208,1.000000e+00,0.022535,0.150964,0.312376,0.253758,0.054811,0.054088,0.054171,...,0.053391,0.993398,0.558463,0.374695,5.934142e-14,0.304700,6.366023e-01,1.000000e+00,0.068360,0.073548
2,0.595197,0.736938,-1.293410e-13,0.238089,0.012249,0.051347,0.063907,0.293184,0.208664,0.092182,...,0.089923,0.001941,0.071647,0.944004,3.741452e-14,0.045293,7.938095e-14,-1.293410e-13,0.140015,0.144938
3,0.238121,0.297500,1.000000e+00,0.597937,0.636603,0.699667,0.691963,0.617180,0.615301,0.614589,...,0.614650,0.993612,0.682552,0.120433,1.000000e+00,0.694789,8.541226e-01,1.000000e+00,0.629159,0.631713
4,0.476171,0.831961,1.000000e+00,0.626985,0.659707,0.650853,0.545918,0.507937,0.499733,0.472216,...,0.471535,0.993498,0.514892,0.323729,2.858824e-14,0.645409,8.646934e-01,1.000000e+00,0.490689,0.493914
5,0.476171,0.831982,1.000000e+00,0.626985,0.665420,0.671023,0.574471,0.507937,0.499986,0.470307,...,0.469766,0.993930,0.528487,0.345639,1.000000e+00,0.666086,8.646934e-01,1.000000e+00,0.485199,0.488462


In [14]:
jet['mode'] = k_cluster.labels_  # jet追加一列

In [15]:
jet_mode = [jet[jet['mode']==n] for n in range(6)]  # jet_mode 

## 2. 仅通过前三个指标进行聚类

In [21]:
k_cluster_2 = cluster.KMeans(n_clusters=6)

In [22]:
train_data_2 = train_data.transpose()[:3].transpose()

In [23]:
k_cluster_2.fit(train_data_2)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=6, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [24]:
pd.DataFrame(k_cluster_2.cluster_centers_)

,0,1,2
0,0.238121,0.297500,1.000000e+00
1,0.999880,0.998201,1.000000e+00
2,0.595197,0.736938,-1.293410e-13
3,0.000036,0.000587,1.000000e+00
4,0.476171,0.831966,1.000000e+00
5,0.833247,0.998219,1.000000e+00


In [25]:
jet['mode2'] = k_cluster_2.labels_

In [26]:
jet_mode_2 = [jet[jet['mode2']==n] for n in range(6)]

In [27]:
import matplotlib.pyplot as plt


def comp_cluster(k):
    plt.figure(figsize=(10, 8))
    for i in range(6):
        plt.hist(jet_mode[i][k], bins=8)
    plt.title(sensor_dict[k])
    plt.show()


def comp_cluster_2(k):
    plt.figure(figsize=(10, 8))
    for i in range(6):
        plt.hist(jet_mode_2[i][k], bins=8)
    plt.title(sensors_dict[k])
    plt.show()

In [ ]:
for i in range(1,25):
    comp_cluster(i)
    comp_cluster_2(i)
    print('_'*50)
#     print('Cluster centers:')
#     print([float(format(k_cluster.cluster_centers_[t][i-1],'.3g')) for t in range(6)])
#     print('-'*100)

可以明显对比出两种方法的不同